In [1]:
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
from sklearn.metrics import average_precision_score, log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split

In [3]:
dirty_train = pd.read_csv("../../../datasets/initial/clean_initial_train.csv").set_index("user_id")
dirty_test = pd.read_csv("../../../datasets/initial/clean_initial_test.csv").set_index("user_id")
#dirty_test.drop("CHURN", axis=1, inplace=True)

In [5]:
def data_load(train_y):
    train_y["REGION"] = train_y["REGION"].astype("category")
    train_y["TENURE"] = train_y["TENURE"].astype("category")
    train_y["MONTANT"] = train_y["MONTANT"].astype("int32")
    train_y["FREQUENCE_RECH"] = train_y["FREQUENCE_RECH"].astype("int16")
    train_y["REVENUE"] = train_y["REVENUE"].astype("int32")
    train_y["FREQUENCE"] = train_y["FREQUENCE"].astype("int16")
    train_y["DATA_VOLUME"] = train_y["DATA_VOLUME"].astype("int32")
    train_y["ON_NET"] = train_y["ON_NET"].astype("int32")
    train_y["ORANGE"] = train_y["ORANGE"].astype("int16")
    train_y["TIGO"] = train_y["TIGO"].astype("int16")  # int16 probably is fine
    train_y["ZONE1"] = train_y["ZONE1"].astype("int16")
    train_y["ZONE2"] = train_y["ZONE2"].astype("int16")
    train_y["REGULARITY"] = train_y["REGULARITY"].astype("int8")
    train_y["FREQ_TOP_PACK"] = train_y["FREQ_TOP_PACK"].astype("int16")

    if "CHURN" in train_y.columns:
        train_y["CHURN"] = train_y["CHURN"].astype("int8")
    return train_y

In [6]:
dirty_X = data_load(dirty_train)
dirty_y = dirty_X["CHURN"]
dirty_X.drop("CHURN", axis=1, inplace=True)

In [7]:
""" 5-split to mix train/valid/generalization valid """
outer_splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [8]:
optuna.logging.set_verbosity(optuna.logging.INFO)

In [9]:
# 5x2 setup
def objective_LGB_dirty(trial):
    results = {}  # dict for record_evaluation callback
    split_iteration = {}  # dict for best iterations in each fold
    inner_valid_values = {}  # dict for valid score inside kfold
    outter_valid_values = {}  # dict for valid score outside kfold
    # simple boosting type gbdt, try another later
    hyperparameters = {
        "objective": "binary",
        "metric": "auc",
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1.0, log=True),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-3, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-3, 10.0),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 8, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.1, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.1, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 0, 30),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 100),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 3, 100),
        "min_sum_hessian_in_leaf": trial.suggest_loguniform(
        "min_sum_hessian_in_leaf", 1e-5, 9e-1
        ),
        "feature_pre_filter": False,
    }
    # Split to 5 sfk manually
    for split_number, data_splits in enumerate(outer_splitter.split(dirty_X, dirty_y)):
        # In single fold split to train/valid manually
        train_inner_X, valid_inner_X, train_inner_y, valid_inner_y = train_test_split(
            dirty_X.reset_index(drop=True).loc[data_splits[0]],
            dirty_y.reset_index(drop=True).loc[data_splits[0]],
            test_size=0.2,
            random_state=1,
            stratify=dirty_y.reset_index(drop=True).loc[data_splits[0]],
        )

        train_set = lgb.Dataset(train_inner_X, label=train_inner_y)
        valid_set = lgb.Dataset(valid_inner_X, label=valid_inner_y)

        # Train
        lgb_model = lgb.train(
            params=hyperparameters,
            train_set=train_set,
            valid_names=["train", "eval"],
            num_boost_round=1000,
            valid_sets=[train_set, valid_set],
            verbose_eval=100,
            early_stopping_rounds=25,
            callbacks=[lgb.record_evaluation(results)],
        )

        split_iteration.update({f"{split_number}": lgb_model.best_iteration})

        inner_valid_values.update({f"{split_number}": results["eval"]["auc"][-1]})

        outter_valid_values.update(
            {
                f"{split_number}": roc_auc_score(
                    y_true=dirty_y.reset_index(drop=True).loc[data_splits[1]],
                    y_score=lgb_model.predict(
                        dirty_X.reset_index(drop=True).loc[data_splits[1]]
                    ),
                )
            }
        )

        print(
            f"Best iteration, split №{split_number} = {lgb_model.best_iteration}\n"
        )  #

    trial.set_user_attr("Best_iteration", split_iteration)

    trial.set_user_attr("Best_inner_valid_score", inner_valid_values)

    trial.set_user_attr("Outter_valid_score", outter_valid_values)

    mean_val = np.mean(list(outter_valid_values.values()))
    trial.set_user_attr("Mean_valid_score", mean_val)

    mean_std = np.std(list(outter_valid_values.values()))
    trial.set_user_attr("Std_valid_score", mean_std)

    trial.set_user_attr("Mean_iterations", np.mean(list(split_iteration.values())))

    print(f"AUC: {mean_val} +\- {mean_std}\n")

    return mean_val

In [11]:
# pruner_LGB_dirty = optuna.pruners.MedianPruner(n_startup_trials=4, n_warmup_steps=100, interval_steps=10)
study_LGB_dirty = optuna.create_study(direction="maximize")  # , pruner=pruner_LGB_dirty
study_LGB_dirty.optimize(objective_LGB_dirty, n_trials=35)

[I 2021-11-01 12:57:45,825] A new study created in memory with name: no-name-f0ecc138-ebfa-4510-a01a-98142556be11


[LightGBM] [Warning] min_data_in_leaf is set=62, min_child_samples=91 will be ignored. Current value: min_data_in_leaf=62
[LightGBM] [Warning] min_data_in_leaf is set=62, min_child_samples=91 will be ignored. Current value: min_data_in_leaf=62
[LightGBM] [Info] Number of positive: 258551, number of negative: 1120039
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022487 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2298
[LightGBM] [Info] Number of data points in the train set: 1378590, number of used features: 14
[LightGBM] [Warning] min_data_in_leaf is set=62, min_child_samples=91 will be ignored. Current value: min_data_in_leaf=62


D:\Anaconda\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
D:\Anaconda\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187547 -> initscore=-1.466026
[LightGBM] [Info] Start training from score -1.466026
Training until validation scores don't improve for 25 rounds
Early stopping, best iteration is:
[6]	train's auc: 0.927053	eval's auc: 0.927616
Best iteration, split №0 = 6

[LightGBM] [Warning] min_data_in_leaf is set=62, min_child_samples=91 will be ignored. Current value: min_data_in_leaf=62
[LightGBM] [Warning] min_data_in_leaf is set=62, min_child_samples=91 will be ignored. Current value: min_data_in_leaf=62
[LightGBM] [Info] Number of positive: 258551, number of negative: 1120039
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020865 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2228
[LightGBM] [Info] Number of data points in the train set: 1378590, number of used features: 14


D:\Anaconda\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
D:\Anaconda\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


...

[100]	train's auc: 0.932418	eval's auc: 0.931649
Early stopping, best iteration is:
[145]	train's auc: 0.933053	eval's auc: 0.931668
Best iteration, split №3 = 145

[LightGBM] [Warning] min_data_in_leaf is set=38, min_child_samples=91 will be ignored. Current value: min_data_in_leaf=38
[LightGBM] [Warning] min_data_in_leaf is set=38, min_child_samples=91 will be ignored. Current value: min_data_in_leaf=38
[LightGBM] [Info] Number of positive: 258551, number of negative: 1120040
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032585 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2212
[LightGBM] [Info] Number of data points in the train set: 1378591, number of used features: 14
[LightGBM] [Warning] min_data_in_leaf is set=38, min_child_samples=91 will be ignored. Current value: min_data_in_leaf=38


D:\Anaconda\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
D:\Anaconda\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187547 -> initscore=-1.466027
[LightGBM] [Info] Start training from score -1.466027
Training until validation scores don't improve for 25 rounds
[100]	train's auc: 0.932331	eval's auc: 0.931314
Early stopping, best iteration is:
[154]	train's auc: 0.933064	eval's auc: 0.931347


[I 2021-11-01 13:29:23,071] Trial 34 finished with value: 0.9312789536743757 and parameters: {'learning_rate': 0.0481019986702336, 'lambda_l1': 0.059264855114262965, 'lambda_l2': 0.016255338228817206, 'min_gain_to_split': 0.9227772804177101, 'num_leaves': 108, 'feature_fraction': 0.8786477385299386, 'bagging_fraction': 0.42644497243428586, 'bagging_freq': 28, 'min_child_samples': 91, 'min_data_in_leaf': 38, 'min_sum_hessian_in_leaf': 0.0029561032405570524}. Best is trial 12 with value: 0.9313126447876939.


Best iteration, split №4 = 154

AUC: 0.9312789536743757 +\- 0.0003180738377547831



In [12]:
study_LGB_dirty.best_trial.user_attrs

{'Best_iteration': {'0': 38, '1': 34, '2': 44, '3': 27, '4': 42},
 'Best_inner_valid_score': {'0': 0.932028145133974,
  '1': 0.9313627125999429,
  '2': 0.9313154095471028,
  '3': 0.931673884564503,
  '4': 0.9313604438364602},
 'Outter_valid_score': {'0': 0.9313291754169085,
  '1': 0.9309622483529927,
  '2': 0.9317045684515295,
  '3': 0.9309599137974409,
  '4': 0.9316073179195978},
 'Mean_valid_score': 0.9313126447876939,
 'Std_valid_score': 0.00031238005300556246,
 'Mean_iterations': 37.0}

In [13]:
study_LGB_dirty.best_params

{'learning_rate': 0.19306546789908532,
 'lambda_l1': 0.009187010220585672,
 'lambda_l2': 0.0010574604018589982,
 'min_gain_to_split': 3.2089796609418078,
 'num_leaves': 100,
 'feature_fraction': 0.82091236456046,
 'bagging_fraction': 0.8347432060517834,
 'bagging_freq': 0,
 'min_child_samples': 47,
 'min_data_in_leaf': 40,
 'min_sum_hessian_in_leaf': 0.40948104553532794}

In [14]:
parameters_for_fit_LGB_dirty = {
    "num_threads": 8,
    "objective": "binary",
    "metric": "auc",
    "feature_pre_filter": False,
    **study_LGB_dirty.best_params,
}

In [15]:
"""2 additional splitters for stacking"""
rs2_splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
rs3_splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)

In [33]:
probs_LGB = pd.DataFrame(
    index=dirty_X.index,
    columns=["LGB_proba_outer", "LGB_proba_rs2", "LGB_proba_rs3", "LGB_proba_mean"],
)

In [59]:
for train_split, predict_split in outer_splitter.split(dirty_X, dirty_y):
    ###############
    # train_set = lgb.Dataset(data=dirty_X.reset_index(drop=True).loc[train_split],
    #                        label=dirty_y.reset_index(drop=True).loc[train_split])
    #
    # predict_set = lgb.Dataset(data=dirty_X.reset_index(drop=True).loc[predict_split],
    #                          label=dirty_y.reset_index(drop=True).loc[predict_split])
    ################
    po = lgb.train(
        parameters_for_fit_LGB_dirty,
        train_set=lgb.Dataset(
            data=dirty_X.reset_index(drop=True).loc[train_split],
            label=dirty_y.reset_index(drop=True).loc[train_split],
        ),
        verbose_eval=False,
        num_boost_round=int(
            np.rint(study_LGB_dirty.best_trial.user_attrs["Mean_iterations"])
        ),
    )
    probs_LGB.loc[
        dirty_X.reset_index().loc[predict_split, "user_id"], "LGB_proba_outer"
    ] = po.predict(data=dirty_X.reset_index(drop=True).loc[predict_split])
print("Outer splitter probs\n", probs_LGB["LGB_proba_outer"].head())

[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Info] Number of positive: 323189, number of negative: 1400049
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2226
[LightGBM] [Info] Number of data points in the train set: 1723238, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187548 -> initscore=-1.466025
[LightGBM] [Info] Start training from score -1.466025
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

In [62]:
for train_split, predict_split in rs2_splitter.split(dirty_X, dirty_y):
    po = lgb.train(
        parameters_for_fit_LGB_dirty,
        train_set=lgb.Dataset(
            data=dirty_X.reset_index(drop=True).loc[train_split],
            label=dirty_y.reset_index(drop=True).loc[train_split],
        ),
        verbose_eval=False,
        num_boost_round=int(
            np.rint(study_LGB_dirty.best_trial.user_attrs["Mean_iterations"])
        ),
    )
    probs_LGB.loc[
        dirty_X.reset_index().loc[predict_split, "user_id"], "LGB_proba_rs2"
    ] = po.predict(data=dirty_X.reset_index(drop=True).loc[predict_split])
print("rs2 splitter probs\n", probs_LGB["LGB_proba_rs2"].head())

[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Info] Number of positive: 323189, number of negative: 1400049
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034675 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2232
[LightGBM] [Info] Number of data points in the train set: 1723238, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187548 -> initscore=-1.466025
[LightGBM] [Info] Start training from score -1.466025
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

In [63]:
for train_split, predict_split in rs3_splitter.split(dirty_X, dirty_y):
    po = lgb.train(
        parameters_for_fit_LGB_dirty,
        train_set=lgb.Dataset(
            data=dirty_X.reset_index(drop=True).loc[train_split],
            label=dirty_y.reset_index(drop=True).loc[train_split],
        ),
        verbose_eval=False,
        num_boost_round=int(
            np.rint(study_LGB_dirty.best_trial.user_attrs["Mean_iterations"])
        ),
    )
    probs_LGB.loc[
        dirty_X.reset_index().loc[predict_split, "user_id"], "LGB_proba_rs3"
    ] = po.predict(data=dirty_X.reset_index(drop=True).loc[predict_split])
print("rs3 splitter probs\n", probs_LGB["LGB_proba_rs3"].head())

[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Info] Number of positive: 323189, number of negative: 1400049
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026097 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2225
[LightGBM] [Info] Number of data points in the train set: 1723238, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187548 -> initscore=-1.466025
[LightGBM] [Info] Start training from score -1.466025
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

In [65]:
probs_LGB["LGB_proba_mean"] = np.mean(probs_LGB[["LGB_proba_outer", "LGB_proba_rs2", "LGB_proba_rs3"]], axis=1)

In [68]:
probs_LGB.drop(["LGB_proba_outer", "LGB_proba_rs2", "LGB_proba_rs3"], axis=1, inplace=True)

In [69]:
probs_LGB.to_csv("LGB_initial_train_OOF.csv", index=False)

In [70]:
refit_lgb_dirty = lgb.train(
    parameters_for_fit_LGB_dirty,
    train_set=lgb.Dataset(dirty_X, label=dirty_y),
    verbose_eval=False,
    num_boost_round=int(np.rint(study_LGB_dirty.best_trial.user_attrs["Mean_iterations"])),
)

[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Info] Number of positive: 403986, number of negative: 1750062
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042170 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2303
[LightGBM] [Info] Number of data points in the train set: 2154048, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.187547 -> initscore=-1.466026
[LightGBM] [Info] Start training from score -1.466026
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

In [71]:
dirty_test = data_load(dirty_test)

In [72]:
subm_dirty = refit_lgb_dirty.predict(dirty_test)

In [73]:
sdf_dirty = pd.DataFrame(columns=["user_id", "CHURN"])
sdf_dirty["user_id"] = dirty_test.index
sdf_dirty["CHURN"] = subm_dirty

In [74]:
sdf_dirty.to_csv("LGB_initial_test.csv", index=False)